In [1]:
# configuration parameters
class Config:
    def __init__(self):
        self.INITIAL_POPULATION_SIZE = 10 # 150
        self.MAX_POPULATION_SIZE = 150
        self.INPUTS = 4
        self.OUTPUTS = 2
        self.MAX_GENERATIONS = 100
        
        self.OFFSPRING_SIZE = 2
        
        self.MUTATE_WEIGHTS_PROPORTION = 0.75
        self.RANDOM_SEED = 42
        self.ENV_RANDOM_SEED = 42
        
CONFIG = Config()

import numpy as np
np.random.seed(CONFIG.RANDOM_SEED)


In [2]:
# TODO: incoming_edge can be a list of innovation numbers

NEURON_TYPES = ["input", "output", "hidden"]

class Neuron:
    NEURON_ID = 0
    
    def __init__(self, neuron_type = None):
        self.type = neuron_type
        if neuron_type == None:
            self.type = 'hidden'
            
        self.incoming_edges = []
        self.value = 0.0
        self.id = type(self).NEURON_ID
        type(self).NEURON_ID += 1
    
    def reset(self):
        self.value = 0.0
    
    def __repr__(self):
        return str(self.__dict__)

In [3]:
# Edge between 2 neurons

class Gene:
    INNOVATION_NUMBER = 0
    
    def __init__(self, input = 0, output = 0, weight = 0.0, enabled = True):
        self.input = input
        self.output = output
        self.weight = weight
        self.enabled = enabled
        self.innovation = type(self).INNOVATION_NUMBER
        type(self).INNOVATION_NUMBER += 1
    
    def __repr__(self):
        return str(self.__dict__)
    
    def mutate(self, epsilon = 1.0):
        self.weight = np.random.normal(self.weight, epsilon)

In [4]:
from math import exp
import pprint

# from config.py import *

# The neural network itself, consisting of neurons and edges (a.k.a genes)
class Genome:
    def __init__(self, neurons = [], genes = []):
        self.neurons = neurons
        self.genes = genes # list of edges
        self.fitness = 0
    
    def __repr__(self):
#         return str(self.__dict__)
        return pprint.PrettyPrinter(indent=4).pformat(self.__dict__)
    
    @staticmethod
    def generate():
        input_neurons = [Neuron('input') for _ in range(CONFIG.INPUTS)]
        output_neurons = [Neuron('output') for _ in range(CONFIG.OUTPUTS)]

        neurons = input_neurons + output_neurons
        
        genes = [Gene(input=input_neuron_index, output=CONFIG.INPUTS + output_neuron_index, weight=np.random.randn())
                 for input_neuron_index, _ in enumerate(input_neurons)
                 for output_neuron_index, _ in enumerate(output_neurons)]
        
        for output_neuron_index, output_neuron in enumerate(output_neurons):
            output_neuron.incoming_edges = [gene_index for gene_index, gene in enumerate(genes)
                                            if gene.output == output_neuron_index]

        # TODO: add random weights
            
        return Genome(neurons=neurons, genes=genes)
    
    @staticmethod
    def sigmoid(x):
#         return 2 / (1 + exp(-4.9 * x)) - 1
        return 1 / (1 + exp(-4.9 * x))

    def evaluate_neuron(self, neuron):
        if neuron.value != 0.0:
            return neuron.value
        
        neuron.value = self.sigmoid(sum(
            [self.genes[edge_index].weight * self.evaluate_neuron(self.neurons[self.genes[edge_index].input])
             for edge_index in neuron.incoming_edges
             if self.genes[edge_index].enabled]
        ))
        return neuron.value

    def activate(self, input_values):
        if len(input_values) != CONFIG.INPUTS:
            raise Error("invalid inputs length of {}".format(len(input_values)))
        
        input_index = 0
        for neuron in self.neurons:
            if neuron.type == 'input':
                neuron.value = input_values[input_index]
                input_index += 1
            else:
                neuron.reset()
        
        output_activations = [self.evaluate_neuron(neuron)
                              for neuron in self.neurons
                              if neuron.type == 'output']
        return output_activations
    
    def mutate_weights(self, proportion = CONFIG.MUTATE_WEIGHTS_PROPORTION):
        selected_genes = np.random.choice(self.genes, int(np.floor(len(self.genes) * proportion)))
        for gene in selected_genes:
            gene.mutate()
    
    # Mutation which adds a new neuron on an existing link between 2 other neurons
    def add_neuron(self):
        # Get a random gene (connection)
        gene = np.random.choice(self.genes)
        
        # take its input neuron
        input_neuron = self.neurons[gene.input]
        
        # take its output neuron
        output_neuron = self.neurons[gene.output]
        
        # create a new neuron
        new_neuron_index = len(self.neurons)
        new_neuron = Neuron()
        self.neurons.append(new_neuron)
        
        assert(self.neurons[len(self.neurons) - 1].id == new_neuron.id)
        
        # create a link from input to new
        link1_index = len(self.genes)
        link1 = Gene(input=gene.input, output=new_neuron_index, weight=1)
        self.genes.append(link1)
        
        # create a link from new to output
        link2_index = len(self.genes)
        link2 = Gene(input=new_neuron_index, output=gene.output, weight=gene.weight)
        self.genes.append(link2)

        # connect everything
        new_neuron.incoming_edges = [link1_index]
        output_neuron.incoming_edges.append(link2_index)
        
        # disable link from input to output
        gene.enabled = False

In [5]:
class Population:
    def __init__(self):
        self.genomes = [Genome.generate() for _ in range(CONFIG.INITIAL_POPULATION_SIZE)]
    
    def __repr__(self):
        return str(self.__dict__)

In [6]:
population = Population()

genome = population.genomes[0]
genome.activate([np.random.randn() for _ in range(CONFIG.INPUTS)])

[0.5, 0.5]

In [7]:
genome.mutate_weights()
genome

{   'fitness': 0,
    'genes': [   {'input': 0, 'output': 4, 'weight': 0.4967141530112327, 'enabled': True, 'innovation': 0},
                 {'input': 0, 'output': 5, 'weight': -0.13826430117118466, 'enabled': True, 'innovation': 1},
                 {'input': 1, 'output': 4, 'weight': 2.195103965605302, 'enabled': True, 'innovation': 2},
                 {'input': 1, 'output': 5, 'weight': 2.0244781879328304, 'enabled': True, 'innovation': 3},
                 {'input': 2, 'output': 4, 'weight': 0.9240332782086917, 'enabled': True, 'innovation': 4},
                 {'input': 2, 'output': 5, 'weight': -0.23413695694918055, 'enabled': True, 'innovation': 5},
                 {'input': 3, 'output': 4, 'weight': 1.8937257113705632, 'enabled': True, 'innovation': 6},
                 {'input': 3, 'output': 5, 'weight': 1.0245515992091883, 'enabled': True, 'innovation': 7}],
    'neurons': [   {'type': 'input', 'incoming_edges': [], 'value': -0.21967188783751193, 'id': 0},
              

In [8]:
genome.add_neuron()

In [9]:
genome

{   'fitness': 0,
    'genes': [   {'input': 0, 'output': 4, 'weight': 0.4967141530112327, 'enabled': True, 'innovation': 0},
                 {'input': 0, 'output': 5, 'weight': -0.13826430117118466, 'enabled': True, 'innovation': 1},
                 {'input': 1, 'output': 4, 'weight': 2.195103965605302, 'enabled': True, 'innovation': 2},
                 {'input': 1, 'output': 5, 'weight': 2.0244781879328304, 'enabled': False, 'innovation': 3},
                 {'input': 2, 'output': 4, 'weight': 0.9240332782086917, 'enabled': True, 'innovation': 4},
                 {'input': 2, 'output': 5, 'weight': -0.23413695694918055, 'enabled': True, 'innovation': 5},
                 {'input': 3, 'output': 4, 'weight': 1.8937257113705632, 'enabled': True, 'innovation': 6},
                 {'input': 3, 'output': 5, 'weight': 1.0245515992091883, 'enabled': True, 'innovation': 7},
                 {'input': 1, 'output': 6, 'weight': 1, 'enabled': True, 'innovation': 80},
                 {'inp

In [10]:
# n1 = Neuron('hidden', [], 1)
# n1.value

np.random.normal(10, 1)
np.random.choice([1, 2, 3], 2)

array([1, 1])

In [11]:
from operator import itemgetter
from copy import deepcopy
import gym
import time


gym.envs.register(
    id='CartPole-v2',
    entry_point='gym.envs.classic_control:CartPoleEnv',
    tags={'wrapper_config.TimeLimit.max_episode_steps': 5000},
    reward_threshold=4750.0,
)

# https://github.com/openai/gym/wiki/CartPole-v0
env = gym.make('CartPole-v2')
env.seed(CONFIG.ENV_RANDOM_SEED)

# print(gym.envs.registry.all())

population = Population()

def fitness_fn(genome, lifespan):
    return lifespan / len(genome.genes)

def one_generation(render):
    lifespans = []
    
    for genome in population.genomes:
        observation = env.reset()
        done = False
        lifespan = 0

        genome.mutate_weights()
        
        while not done:
            if render:
                env.render()
            
            lifespan = lifespan + 1
            activation = genome.activate(observation)
            action = np.argmax(activation)
            observation, reward, done, info = env.step(action)
            
            if done:
                genome.fitness = fitness_fn(genome, lifespan)
                lifespans.append(lifespan)
                #print("Episode finished after {} timestamps".format(lifespan))
    
    # sort by fitness
    population.genomes.sort(key=lambda genome: genome.fitness, reverse=True)
    
    print("Lifespans: {}".format(sorted(lifespans, reverse=True)))
    print(list(map(lambda genome: genome.fitness, population.genomes)))
    
    # always mutate weights, but the better the individual smaller the weight mutation
    
    # create offspring of the top N genomes by adding a random neuron
    new_genomes = []
    for genome in population.genomes[:CONFIG.OFFSPRING_SIZE]:
        copy = deepcopy(genome)
        copy.add_neuron()
        new_genomes.append(copy)
    
    # survival of the fittest
    population.genomes = (new_genomes + population.genomes)[:CONFIG.MAX_POPULATION_SIZE]


def run_neuroevolution():
    for generation in range(CONFIG.MAX_GENERATIONS):
        is_last_generation = generation == CONFIG.MAX_GENERATIONS - 1
        print("Genration: {}".format(generation))
        one_generation(render=is_last_generation)

        if is_last_generation:
            print(list(map(lambda genome: genome.fitness, population.genomes)))

run_neuroevolution()
env.close()
population

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Genration: 0
Lifespans: [10, 10, 10, 10, 10, 10, 10, 9, 8, 8]
[1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.125, 1.0, 1.0]
Genration: 1
Lifespans: [11, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 8]
[1.375, 1.25, 1.25, 1.25, 1.25, 1.25, 1.125, 1.125, 1.125, 1.0, 1.0, 1.0]
Genration: 2
Lifespans: [11, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 8, 8]
[1.375, 1.25, 1.25, 1.25, 1.125, 1.125, 1.125, 1.125, 1.125, 1.0, 1.0, 1.0, 0.9, 0.8]
Genration: 3
Lifespans: [11, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 8, 8, 8]
[1.25, 1.25, 1.25, 1.25, 1.125, 1.125, 1.125, 1.125, 1.1, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 0.8]
Genration: 4
Lifespans: [11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 8, 8, 8, 8]
[1.375, 1.25, 1.25, 1.25, 1.25, 1.125, 1.125, 1.125, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 0.8, 0.8]
Genration: 5
Lifespans: [11, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8]
[1.375, 1.25, 1.25,

Lifespans: [144, 121, 77, 34, 29, 28, 14, 14, 12, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8]
[12.0, 7.5625, 5.5, 2.4285714285714284, 2.4166666666666665, 1.75, 1.4, 1.25, 1.2, 1.125, 1.125, 1.125, 1.125, 1.125, 1.125, 1.125, 1.125, 1.1, 1.1, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.8333333333333334, 0.8333333333333334, 0.8333333333333334, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.75, 0.6666666666666666, 0.6666666666666666, 0.6428571428571429, 0.6428571428571429]
Genration: 32
Lifespans: [315, 142, 110, 92, 42, 36, 23, 21, 15, 14, 13, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 

Lifespans: [1996, 230, 162, 160, 156, 142, 138, 72, 69, 60, 58, 58, 45, 26, 18, 14, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
[99.8, 14.375, 10.0, 9.857142857142858, 9.75, 9.0, 8.875, 4.833333333333333, 4.833333333333333, 4.5, 3.75, 3.1363636363636362, 2.5, 1.8571428571428572, 1.8, 1.4, 1.25, 1.25, 1.25, 1.25, 1.125, 1.125, 1.125, 1.125, 1.125, 1.125, 1.1, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.8333333333333334, 0.8333333333333334, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.7857142857142857, 0.75, 0.75, 0.7142857142857143, 0.7142857142857143, 0.6666666666666666, 0.6666666666666666, 0.6428571428571429, 0.6428571428571429, 0.625, 0.6

Lifespans: [2046, 451, 308, 279, 161, 154, 153, 152, 150, 147, 144, 131, 125, 101, 56, 50, 45, 29, 28, 27, 22, 16, 14, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8]
[93.0, 18.791666666666668, 17.4375, 12.833333333333334, 9.375, 9.0, 8.5, 8.444444444444445, 8.166666666666666, 7.7, 7.318181818181818, 7.214285714285714, 6.944444444444445, 5.954545454545454, 3.5714285714285716, 2.8125, 2.4166666666666665, 2.2, 2.1538461538461537, 1.6, 1.5555555555555556, 1.35, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.125, 1.125, 1.125, 1.125, 1.1, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.8333333333333334, 0.8333333333

Lifespans: [5000, 2632, 1179, 345, 214, 212, 200, 186, 173, 154, 152, 132, 125, 79, 66, 57, 47, 47, 45, 33, 33, 24, 22, 21, 13, 13, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8]
[192.30769230769232, 119.63636363636364, 49.125, 14.375, 11.777777777777779, 10.8125, 10.333333333333334, 8.333333333333334, 8.23076923076923, 7.6, 6.0, 5.923076923076923, 5.208333333333333, 4.071428571428571, 3.0384615384615383, 2.8125, 2.611111111111111, 2.611111111111111, 2.5384615384615383, 2.357142857142857, 2.1, 2.0, 1.8333333333333333, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.125, 1.125, 1.125, 1.1, 1.1, 1.1, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9, 0.9

...

Lifespans: [5000, 5000, 5000, 5000, 4398, 3464, 3084, 2183, 1632, 847, 350, 249, 233, 216, 209, 199, 175, 169, 166, 165, 164, 148, 147, 145, 141, 105, 88, 85, 85, 84, 84, 83, 67, 62, 59, 54, 50, 47, 45, 45, 44, 40, 40, 39, 37, 27, 27, 26, 26, 25, 24, 21, 19, 18, 14, 12, 12, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
[208.33333333333334, 208.33333333333334, 169.15384615384616, 156.25, 156.25, 115.46666666666667, 90.70588235294117, 64.20588235294117, 51.0, 28.233333333333334, 21.875, 10.8, 9.25, 9.166666666666666, 9.11111111111111, 8.8125, 8.3, 8.038461538461538, 7.766666666666667, 7.653846153846154, 7.291666666666667, 6.384615384615385, 6.035714285714286, 4.9, 4.833333333333333, 4.722222222222222, 3.357142

Lifespans: [5000, 5000, 5000, 5000, 2096, 1430, 452, 273, 253, 216, 213, 209, 196, 179, 171, 164, 164, 158, 157, 149, 143, 139, 136, 131, 125, 125, 123, 116, 114, 114, 110, 108, 97, 92, 92, 82, 80, 79, 69, 63, 61, 57, 50, 45, 42, 41, 38, 38, 35, 31, 29, 29, 28, 26, 26, 15, 14, 13, 13, 12, 11, 11, 11, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
[166.66666666666666, 166.66666666666666, 156.25, 147.05882352941177, 58.22222222222222, 39.72222222222222, 17.384615384615383, 15.8125, 12.0, 10.88888888888889, 8.53125, 8.2, 7.277777777777778, 7.1, 6.833333333333333, 6.147058823529412, 5.7, 5.681818181818182, 5.642857142857143, 5.59375, 5.107142857142857, 4.966666666666667, 4.9375, 4.857142857142857, 4.361111111111111, 4.34375, 4.3125, 4

Lifespans: [5000, 5000, 3794, 3524, 3316, 2106, 438, 396, 327, 235, 227, 226, 216, 203, 202, 188, 181, 175, 168, 166, 166, 164, 163, 160, 157, 152, 151, 151, 144, 140, 139, 138, 106, 104, 103, 95, 91, 78, 65, 64, 64, 52, 52, 50, 50, 44, 44, 41, 40, 38, 35, 33, 31, 31, 29, 27, 26, 26, 26, 24, 21, 20, 19, 16, 13, 13, 12, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8]
[166.66666666666666, 131.57894736842104, 118.5625, 110.53333333333333, 110.125, 75.21428571428571, 24.75, 14.6, 13.055555555555555, 10.9, 9.8125, 9.416666666666666, 9.222222222222221, 8.73076923076923, 8.6875, 8.38888888888889, 7.769230769230769, 7.6, 6.766666666666667, 6.352941176470588, 6.25, 6.153846153846154, 6.033333333333333, 5.875, 5.6875, 5.466666666666667, 5.416666666666667, 5.1875, 

Lifespans: [5000, 5000, 3991, 3592, 3424, 3208, 3077, 2828, 2194, 317, 313, 263, 228, 228, 215, 188, 187, 175, 163, 158, 157, 155, 152, 141, 139, 133, 131, 130, 127, 125, 119, 104, 99, 91, 88, 83, 81, 79, 70, 63, 57, 51, 50, 50, 50, 46, 44, 43, 38, 38, 35, 31, 30, 20, 18, 16, 16, 16, 15, 14, 13, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8]
[312.5, 166.66666666666666, 142.53571428571428, 119.73333333333333, 107.0, 106.93333333333334, 90.5, 88.375, 73.13333333333334, 19.5625, 9.90625, 8.444444444444445, 7.833333333333333, 7.8125, 7.75, 7.6, 7.388888888888889, 7.125, 6.71875, 6.583333333333333, 6.575, 6.541666666666667, 6.266666666666667, 6.233333333333333, 5.46875, 5.09375, 5.0, 4.964285714285714, 4.884615384615385, 4.

KeyboardInterrupt: 